In [72]:
import madmom
import librosa
#import pyrubberband
from IPython import display
import numpy as np
import soundfile

In [73]:
def repeat_track(src_file, tgt_file, n_repeats):
    y, sr = librosa.load(src_file)
    y = np.concatenate([y] * n_repeats)
    soundfile.write(tgt_file, y, sr)

In [74]:
repeat_track('drum_track3.wav', 'drum_track3_long.wav', 20)

In [79]:
CELLO_TRACK = 'cello_track.wav'
# DRUM_TRACK = 'drum_track3_long.wav'
DRUM_TRACK = 'beatbox.wav'

In [76]:
y_cello, sr_cello = librosa.load(CELLO_TRACK)

In [80]:
y_drum, sr_drum = librosa.load(DRUM_TRACK)
y_drum_short = None

In [61]:
# n_repeats = 80
# n_bars_short = 6
# n_bars = n_bars_short * n_repeats
# y_drum = np.concatenate([y_drum] * n_repeats)
# drum_bar_length = len(y_drum) / (n_bars) / sr_drum
# downbeats_drum_all = np.array([i * drum_bar_length for i in range(n_bars)])

In [142]:
# with open('madmom_out/cello.downbeats') as f:
#     downbeats_cello = [float(x) for x in f]
# with open('madmom_out/beatbox.downbeats') as f:
#     downbeats_drum = [float(x) for x in f]
# downbeats_cello = downbeats_cello[:len(downbeats_drum)]

In [62]:
def get_downbeats(filename):
    in_processor = madmom.features.downbeats.RNNDownBeatProcessor()
    out_processor = madmom.features.downbeats.DBNDownBeatTrackingProcessor(beats_per_bar=[3, 4], fps=100)
    result = out_processor(in_processor(filename))
    return result[np.isclose(result[:, 1], 1.), 0]

In [81]:
downbeats_drum = get_downbeats(DRUM_TRACK)
downbeats_cello = get_downbeats(CELLO_TRACK)

In [ ]:
# Remove every other drum downbeat to make the track twice as fast
# downbeats_drum = downbeats_drum[::2]

In [82]:
downbeats_cello = downbeats_cello[:len(downbeats_drum)]

In [69]:
def mix_tracks(tracks, weights=None):
    max_len = max(len(track) for track in tracks)
    tracks = [np.pad(track, (0, max_len - len(track))) for track in tracks]
    return np.average(tracks, axis=0, weights=weights)

In [ ]:
def warp(audio, sr, times, target_times):
    assert len(target_times) <= len(times)

    result = [np.zeros([int(target_times[0] * sr)], np.float32)]

    for i in range(len(target_times) - 1):
        start = times[i]
        end = times[i + 1]
        target_start = target_times[i]
        target_end = target_times[i + 1]

        rate = (end - start) / (target_end - target_start)
        y = audio[int(start * sr): int(end * sr)]

        y_stretched = librosa.effects.time_stretch(y, rate)
        #y_stretched = pyrubberband.pyrb.time_stretch(y, sr=sr_cello, rate=rate)

        result.append(y_stretched)

    return np.concatenate(result)

In [ ]:
y_drums_warped = warp(y_drum, sr_drum, downbeats_drum[::2], downbeats_cello[:len(downbeats_drum[::2])])
output = mix_tracks([y_drums_warped, y_cello], weights=[0.5, 0.5])
display.display(display.Audio(output, rate=sr_drum))

In [87]:
soundfile.write(f'output_{CELLO_TRACK}_{DRUM_TRACK}_2x.wav', output, sr_drum)